In [ ]:
%matplotlib widget
import scipp as sc

from ess.estia.load import load_mcstas_events
from ess.estia import EstiaWorkflow
from ess.reflectometry.types import *
from ess.reflectometry.supermirror import CriticalEdge


wf = EstiaWorkflow()
wf.insert(load_mcstas_events)

wf[YIndexLimits]  = sc.scalar(35), sc.scalar(64)
wf[ZIndexLimits] = sc.scalar(0), sc.scalar(14 * 32)
wf[BeamDivergenceLimits] = sc.scalar(-0.75, unit='deg'), sc.scalar(0.75, unit='deg')
wf[WavelengthBins] = sc.geomspace('wavelength', 3.5, 12, 301, unit='angstrom')
wf[QBins] = sc.linspace('Q', 0.001, 0.1, 100, unit='1/angstrom')
# Reference sample is perfect supermirror with reflectivity = 1 everywhere
wf[CriticalEdge] = 100 * wf.compute(CriticalEdge)

# There is no proton current data in the McStas files, here we just add some fake proton current
# data to make the workflow run.
wf[ProtonCurrent[SampleRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})
wf[ProtonCurrent[ReferenceRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})

In [ ]:
first = 221229

run_numbers = dict(  # noqa: C408
    reference=list(range(first, first+4)),
    magnetic_reference=list(range(first+4, first+8)),
)

first = 221241
run_numbers |= dict(  # noqa: C408
    sample1=list(range(first, first+4)),
    sample2=list(range(first+4, first+8)),
)

def path(num):
    return f'polarization-runs/{num}/tof_detector_moved_list.p.x.y.t.L.sx.sy.sz.U1'


In [ ]:
from ess.estia.calibration import reflectivity_provider
from ess.estia.types import *


results = {}
for run in ('reference', 'magnetic_reference'):
    for pth in map(path, run_numbers[run]):
        w = wf.copy()
        w[Filename[SampleRun]] = path(run_numbers['sample1'][0])
        w[Filename[ReferenceRun]] = pth
        results.setdefault(run, []).append(w.compute(Reference))


#for pth in map(path, run_numbers['sample1']):
for pth in map(path, run_numbers['magnetic_reference']):
    w = wf.copy()
    w[Filename[SampleRun]] = pth
    w[Filename[ReferenceRun]] = path(run_numbers['reference'][0])
    results.setdefault('sample1', []).append(w.compute(Sample))

# Some bookkeeping necessary to make Scipp drop coords
# instead of raising errors
for res in results.values():
    for r in res:
        for c in r.coords:
            r.coords.set_aligned(c, False)

w = wf.copy()
w.insert(reflectivity_provider)
w[Intensity[NonMagneticReference, OffOff]] = results['reference'][0]
w[Intensity[NonMagneticReference, OffOn]] = results['reference'][1]
w[Intensity[NonMagneticReference, OnOff]] = results['reference'][2]
w[Intensity[NonMagneticReference, OnOn]] = results['reference'][3]

w[Intensity[MagneticReference, OffOff]] = results['magnetic_reference'][0]
w[Intensity[MagneticReference, OffOn]] = results['magnetic_reference'][1]
w[Intensity[MagneticReference, OnOff]] = results['magnetic_reference'][2]
w[Intensity[MagneticReference, OnOn]] = results['magnetic_reference'][3]

w[Intensity[MagneticSample, OffOff]] = results['sample1'][0]
w[Intensity[MagneticSample, OffOn]] = results['sample1'][1]
w[Intensity[MagneticSample, OnOff]] = results['sample1'][2]
w[Intensity[MagneticSample, OnOn]] = results['sample1'][3]

In [ ]:
R = w.compute(PolarizedReflectivityOverQ)

In [ ]:
with open('SNAG_m5p5_Tdown.ref') as f:
    d = [v.strip().split('\t') for v in f.readlines()][:-1]

R_true = sc.DataArray(data=sc.array(dims='Q',  values=[float(v[1]) for v in d]), coords={'Q': sc.array(dims='Q', values=[float(v[0]) for v in d], unit='1/angstrom')})

In [ ]:
R_without_correction = []
for i in range(4):
    w = wf.copy()
    #w[Filename[SampleRun]] = path(run_numbers['sample1'][i])
    w[Filename[SampleRun]] = path(run_numbers['magnetic_reference'][i])
    w[Filename[ReferenceRun]] = path(run_numbers['reference'][i])
    R_without_correction.append(w.compute(ReflectivityOverQ).hist())

In [ ]:
import plopp as pp
from ess.reflectometry.normalization import reduce_from_lz_to_q
labels = ['22', '23', '32', '33']

p = pp.plot(
    {
        f'pa={labels[i]}': reduce_from_lz_to_q(r, w.compute(QBins)) for i, r in enumerate(results['magnetic_reference'])
    },
    title='Magnetic reference',
    norm='log'
)
p.save('polarization-figures/magnetic-reference.png')
p

In [ ]:
p = pp.plot(
    {
        f'pa={labels[i]}': reduce_from_lz_to_q(r, w.compute(QBins)) for i, r in enumerate(results['reference'])
    },
    title='Reference',
    norm='log'
)
p.save('polarization-figures/reference.png')
p

In [ ]:
p = pp.plot(
    {
        f'pa={labels[i]}': reduce_from_lz_to_q(r, w.compute(QBins)) for i, r in enumerate(results['sample1'])
    },
    title='Magnetic sample',
    norm='log'
)
p.save('polarization-figures/sample_a1.png')
p

In [ ]:
labels = ['R down down', 'R down up', 'R up down', 'R up up']

p = pp.plot(
    {
        f'{labels[i]}': r for i, r in enumerate(R)
    } | {'true': R_true},
    title='Reflectivity, corrected',
    norm='log'
)
p.save('polarization-figures/reflectivity_corrected.png')
p

In [ ]:
p = pp.plot(
    {
        f'{labels[i]}': r for i, r in enumerate(R_without_correction)
    } | {'true': R_true},
    title='Reflectivity, not corrected',
    norm='log'
)
p.save('polarization-figures/reflectivity_not_corrected.png')
p